In [347]:
import base64
import datetime
import json
import numbers
import secrets

from dateutil import tz

from blazarclient.client import Client as _BlazarClient # installed from github
from heatclient.client import Client as HeatClient
import keystoneauth1 as ksa
import keystoneauth1.loading
import keystoneauth1.session
from novaclient.client import Client as NovaClient
from novaclient import exceptions as novaexceptions

from hammers import osapi

In [3]:
def auth_from_rc(rc):
    """
    Fun with naming schemes:    
    * envvar name:          OS_AUTH_URL
    * loader option name:      auth-url
    * loader argument name:    auth_url
    """
    rc_opt_keymap = {key[3:].lower().replace('_', '-'): key for key in rc}
    loader = ksa.loading.get_plugin_loader('password')
    credentials = {}
    for opt in loader.get_options():
        if opt.name not in rc_opt_keymap:
            continue
        credentials[opt.name.replace('-', '_')] = rc[rc_opt_keymap[opt.name]]
    auth = loader.load_from_options(**credentials)
    return auth

In [4]:
rc = osapi.load_osrc('../rc/chi.tacc.rc', get_pass=True)
sess = ksa.session.Session(auth=auth_from_rc(rc))

Enter your password: ········


In [30]:
def blazar_client_builder(session):
    bc = BlazarClient('1', 
        blazar_url=session.get_endpoint(service_type='reservation'), 
        auth_token=session.get_token(),
    )
    return bc

In [9]:
nc = NovaClient('2', session=sess)

In [173]:
bc = blazar_client_builder(sess)

In [14]:
name = 'image-builder-{}'.format(base64.b32encode(secrets.token_bytes(6)).decode('ascii').strip('='))
name

'image-builder-LHBNMGV754'

```python
physical_reservations = []
for phys_res_str in parsed_args.physical_reservations:
    err_msg = ("Invalid physical-reservation argument '%s'. "
               "Reservation arguments must be of the "
               "form --physical-reservation <min=int,max=int,"
               "hypervisor_properties=str,resource_properties=str>"
               % phys_res_str)
    phys_res_info = {"min": "", "max": "", "hypervisor_properties": "",
                     "resource_properties": ""}
    prog = re.compile('^(\w+)=(\w+|\[[^]]+\])(?:,(.+))?$')

    def parse_params(params):
        match = prog.search(params)
        if match:
            self.log.info("Matches: %s", match.groups())
            k, v = match.group(1, 2)
            if k in phys_res_info:
                phys_res_info[k] = v
            else:
                raise exception.IncorrectLease(err_msg)
            if len(match.groups()) == 3 and match.group(3) is not None:
                parse_params(match.group(3))

    parse_params(phys_res_str)
    if not phys_res_info['min'] and not phys_res_info['max']:
        raise exception.IncorrectLease(err_msg)
    # NOTE(sbauza): The resource type should be conf-driven mapped with
    #               blazar.conf file but that's potentially on another
    #               host
    phys_res_info['resource_type'] = 'physical:host'
    physical_reservations.append(phys_res_info)
if physical_reservations:
    params['reservations'] += physical_reservations
```

In [87]:
BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

def random_base32(n_bytes):
    tok = secrets.token_bytes(n_bytes)
    return base64.b32encode(tok).decode('ascii').strip('=')

def lease_create_args(name=None, start='now', length=60*60*24, nodes=1, resource_properties=''):
    if name is None:
        name = 'lease-{}'.format(random_base32(6))

    if start == 'now':
        start = datetime.datetime.now(tz=tz.tzutc()) + datetime.timedelta(seconds=70)
    if isinstance(length, numbers.Number):
        length = datetime.timedelta(seconds=length)
    
    end = start + length
        
    if resource_properties:
        resource_properties = json.dumps(resource_properties)

    reservations = [{
        'resource_type': 'physical:host',
        'resource_properties': resource_properties,
        'hypervisor_properties': '',
        'min': str(nodes), 'max': str(nodes),
    }]
        
    query = {
        'name': name,
        'start': start.strftime(BLAZAR_TIME_FORMAT),
        'end': end.strftime(BLAZAR_TIME_FORMAT), 
        'reservations': reservations,
        'events': [],
    }
    return query

In [55]:
node_types = {
    'compute',
    'compute_ib',
    'storage',
    'storage_hierarchy',
    'gpu_p100',
    'gpu_k80',
    'gpu_m40',
    'fpga',
    'lowpower_xeon',
    'atom',
    'arm64',
}

def lease_create_nodetype(*args, **kwargs):
    try:
        node_type = kwargs.pop('node_type')
    except KeyError:
        raise ValueError('no node_type specified')
    if node_type not in node_types:
        raise ValueError('unknown node_type ("{}")'.format(node_type))
    kwargs['resource_properties'] = ['=', '$node_type', node_type]
    return lease_create_args(*args, **kwargs)

In [171]:
args = lease_create_nodetype(node_type='compute')
args

{'end': '2017-04-07 15:19',
 'events': [],
 'name': 'lease-OMZXGUT33Y',
 'reservations': [{'hypervisor_properties': '',
   'max': '1',
   'min': '1',
   'resource_properties': '["=", "$node_type", "compute"]',
   'resource_type': 'physical:host'}],
 'start': '2017-04-06 15:19'}

In [174]:
my_lease = bc.lease.create(**args)

In [182]:
my_lease = bc.lease.get(my_lease['id'])#['status_reason']

In [187]:
my_lease['action'], my_lease['status'] == ('START', 'COMPLETE')

True

In [180]:
my_lease['id']

'813b3c03-8e71-41ec-80e3-94243ea805ee'

In [161]:
nc.images.find(name='CC-CentOS7')

<Image: CC-CentOS7>

In [162]:
nc.images.find(id='CC-CentOS7')

NotFound: No Image matching {'id': 'CC-CentOS7'}. (HTTP 404)

In [163]:
import novaclient

In [193]:
def resolve_image_idname(novaclient, idname):
    try:
        return novaclient.images.find(id=idname)
    except novaexceptions.NotFound:
        return novaclient.images.find(name=idname)

In [189]:
DEFAULT_IMAGE = '0f216b1f-7841-451b-8971-d383364e01a6'
DEFAULT_IMAGE = 'CC-CentOS7'

def instance_create_args(lease, name=None, image=DEFAULT_IMAGE, key=None):
    if name is None:
        name = 'instance-{}'.format(random_base32(6))

    return {
        'name': name,
        'flavor': 'baremetal',
        'image': image,
        # 'reservation_id': lease['reservations'][0]['id'],
        'scheduler_hints': {
            'reservation': lease['reservations'][0]['id'],
        },
        # 'nics': '', # automatically binds one, not needed unless want non-default?
        'key_name': key,
    }

In [91]:
from glanceclient.client import Client as GlanceClient

In [92]:
gc = GlanceClient('1', session=sess)

In [148]:
centos = gc.images.find(name='CC-CentOS7')

In [195]:
inst_image = resolve_image_idname(nc, 'CC-CentOS7')
server_args = instance_create_args(my_lease, key='nick-mbp17', image=inst_image)
print(server_args)
my_instance = nc.servers.create(**server_args)
my_instance

{'name': 'instance-UJMKRQKRGA', 'flavor': 'baremetal', 'image': <Image: CC-CentOS7>, 'scheduler_hints': {'reservation': '7b8b2b6b-ad2f-40b3-a06b-254c7e19b1ce'}, 'key_name': 'nick-mbp17'}


<Server: instance-UJMKRQKRGA>

In [212]:
import uuid
inst_image = resolve_image_idname(nc, 'CC-CentOS7')
fail_args = instance_create_args(my_lease, key='nick-mbp17', image=inst_image)
fail_args['scheduler_hints']['reservation'] = str(uuid.uuid4())
print(fail_args)
my_failinstance = nc.servers.create(**fail_args)
my_failinstance

{'name': 'instance-7NCT33ZHRU', 'flavor': 'baremetal', 'image': <Image: CC-CentOS7>, 'scheduler_hints': {'reservation': 'c90a7f5a-16c5-4b8d-a2b4-004abea8a9c6'}, 'key_name': 'nick-mbp17'}


<Server: instance-7NCT33ZHRU>

In [213]:
my_failinstance.get()

In [216]:
my_failinstance.fault

{'code': 500,
 'created': '2017-04-06T15:53:44Z',
 'message': 'No valid host was found. There are not enough hosts available.'}

In [217]:
my_failinstance.id

'a312e5e2-fc6f-4d15-a0d3-618e45732349'

In [201]:
import datetime
import time

In [204]:
datetime.datetime.now().isoformat()

'2017-04-06T10:32:05.736192'

In [210]:
for _ in range(200):
    time.sleep(10)
    my_instance.get()
    print('{} stat:{} upd:{}'.format(datetime.datetime.now().isoformat(), my_instance.status, my_instance.updated))
    if my_instance.status != 'BUILD':
        break

2017-04-06T10:34:09.943600 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:34:20.576771 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:34:30.819393 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:34:41.161329 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:34:51.502328 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:35:01.762983 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:35:12.085914 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:35:22.430271 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:35:32.692484 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:35:43.009659 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:35:53.251143 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:36:03.599786 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:36:13.939468 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:36:24.195385 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:36:34.415824 stat:BUILD upd:2017-04-06T15:29:52Z
2017-04-06T10:36:44.603758 stat:BUILD upd:2017-04-06T15

In [208]:
my_instance.update()

In [209]:
my_instance.to_dict()

{'OS-DCF:diskConfig': 'MANUAL',
 'OS-EXT-AZ:availability_zone': 'climate:7b8b2b6b-ad2f-40b3-a06b-254c7e19b1ce',
 'OS-EXT-STS:power_state': 0,
 'OS-EXT-STS:task_state': 'spawning',
 'OS-EXT-STS:vm_state': 'building',
 'OS-SRV-USG:launched_at': None,
 'OS-SRV-USG:terminated_at': None,
 'accessIPv4': '',
 'accessIPv6': '',
 'addresses': {},
 'adminPass': 'f8po6erfTLMS',
 'config_drive': '',
 'created': '2017-04-06T15:29:50Z',
 'flavor': {'id': 'baremetal',
  'links': [{'href': 'http://chi.tacc.chameleoncloud.org:8774/975c0a94b784483a885f4503f70af655/flavors/baremetal',
    'rel': 'bookmark'}]},
 'hostId': 'e132f00cc18a9c63324a0dcd0dd5ea19149dc3bab699c79d285d2842',
 'id': '1853fed9-e9a4-4d0b-bf39-8658a5b7ae61',
 'image': {'id': '0f216b1f-7841-451b-8971-d383364e01a6',
  'links': [{'href': 'http://chi.tacc.chameleoncloud.org:8774/975c0a94b784483a885f4503f70af655/images/0f216b1f-7841-451b-8971-d383364e01a6',
    'rel': 'bookmark'}]},
 'key_name': 'nick-mbp17',
 'links': [{'href': 'http://chi.

In [199]:
my_instance.status

'BUILD'

In [143]:
my_instance.delete()

()

In [219]:
my_oob = nc.servers.get('00dcccc7-2ba6-4965-a0ce-3fe901e933c3')

In [242]:
fips = nc.floating_ips.list()

In [243]:
fips

[<FloatingIP fixed_ip=10.40.0.24, id=9bbc404c-aac7-4d57-b987-edb58b48db90, instance_id=d53f5ba0-391f-4fb8-bc3f-779c57b123ae, ip=129.114.108.211, pool=ext-net>]

In [250]:
from neutronclient.v2_0.client import Client as NeutronClient

In [248]:
# from neutronclient.client import SessionClient as NeutronClient

In [251]:
# nec = NeutronClient(session=sess)

In [254]:
# nec.create_floatingip()

In [298]:
free_fips = list(fip for fip in nc.floating_ips.list() if fip.instance_id is None)
free_fips

[<FloatingIP fixed_ip=None, id=017e7471-250c-4f4c-95ea-b25d7f603e51, instance_id=None, ip=129.114.109.146, pool=ext-net>]

In [299]:
for fip in free_fips:
    fip.delete()

In [274]:
def get_create_floatingip(nc):
    created = False
    ips = nc.floating_ips.list()
    unbound = (ip for ip in ips if ip.instance_id is None)
    try:
        fip = next(unbound)
    except StopIteration:
        fip = nc.floating_ips.create('ext-net')
        created = True
    return created, fip

created, fip = get_create_floatingip(nc)

In [301]:
from blazarclient.client import Client as _BlazarClient # installed from github

class BlazarClient(object):
    """
    Current BlazarClient doesn't support sessions, just a token, so it
    behaves poorly after its token expires. This is a thin wrapper that 
    recreates the real client every X minutes to avoid expiration.
    """
    def __init__(self, version, session):
        self._version = version
        self._session = session
        self._client_age = None
        self._create_client()

    def _create_client(self):
        self._bc = _BlazarClient(
            self._version, 
            blazar_url=self._session.get_endpoint(service_type='reservation'), 
            auth_token=self._session.get_token(),
        )
        self._client_age = time.monotonic()
        
    def __getattr__(self, attr):
        if time.monotonic() - self._client_age > 20*60:
            self._create_client()
        return getattr(self._bc, attr)

In [302]:
bc = BlazarClient('1', sess)

In [297]:
fip.delete()

()

In [303]:
bc.lease.delete('820bae2f-a58f-4205-840d-d58494de779a')

In [262]:
# pool = nc.floating_ip_pools.find(name='ext-net')


<FloatingIP fixed_ip=None, id=017e7471-250c-4f4c-95ea-b25d7f603e51, instance_id=None, ip=129.114.109.146, pool=ext-net>

In [277]:
fip.ip

'129.114.108.242'

In [278]:
import fabric

ModuleNotFoundError: No module named 'fabric'

In [273]:
created, fip = get_create_floatingip(nc)


(False,
 <FloatingIP fixed_ip=None, id=35c7794f-c761-410f-8774-e5d8d68b78db, instance_id=None, ip=129.114.108.242, pool=ext-net>)

In [266]:
my_oob.add_floating_ip(fip)

()

In [268]:
fip.get()

In [269]:
fip

<FloatingIP fixed_ip=10.40.0.53, id=017e7471-250c-4f4c-95ea-b25d7f603e51, instance_id=00dcccc7-2ba6-4965-a0ce-3fe901e933c3, ip=129.114.109.146, pool=ext-net>

In [140]:
my_instance.metadata['hello'] = 'world'

In [160]:
my_instance.to_dict()

{'OS-DCF:diskConfig': 'MANUAL',
 'OS-EXT-AZ:availability_zone': 'climate:bf125945-3ae9-4a5a-a21b-ae126eeef7bf',
 'OS-EXT-STS:power_state': 0,
 'OS-EXT-STS:task_state': 'spawning',
 'OS-EXT-STS:vm_state': 'building',
 'OS-SRV-USG:launched_at': None,
 'OS-SRV-USG:terminated_at': None,
 'accessIPv4': '',
 'accessIPv6': '',
 'addresses': {},
 'adminPass': '8upZwdV2UJPY',
 'config_drive': '',
 'created': '2017-04-06T03:00:55Z',
 'flavor': {'id': 'baremetal',
  'links': [{'href': 'http://chi.tacc.chameleoncloud.org:8774/975c0a94b784483a885f4503f70af655/flavors/baremetal',
    'rel': 'bookmark'}]},
 'hostId': '0d0c84f9f20f10c36d934b0fcc4d97dd69dac406ecb69a232efc9341',
 'id': 'c7148cc0-a311-4cf6-b32f-989e289ed74a',
 'image': {'id': '0f216b1f-7841-451b-8971-d383364e01a6',
  'links': [{'href': 'http://chi.tacc.chameleoncloud.org:8774/975c0a94b784483a885f4503f70af655/images/0f216b1f-7841-451b-8971-d383364e01a6',
    'rel': 'bookmark'}]},
 'key_name': 'nick-mbp17',
 'links': [{'href': 'http://chi.

In [70]:
bc.lease.get(my_lease['id'])['status_reason']

'Successfully created lease'

In [75]:
bc.lease.delete(gpu100_1['id'])

In [33]:
nc.servers.create()

In [359]:
my_inst = nc.servers.get('f41f9901-5b12-4cd0-9da5-94cc66b2655a')

In [360]:
gputest = nc.images.find(id='87224200-ea8c-4286-8e2b-e5ac5b089b36')
gputest

<Image: test-CC-Ubuntu16.04-CUDA8>

In [483]:
xenial = nc.images.find(name='CC-Ubuntu16.04')
my_inst.rebuild(xenial)

<Server: image-builder-gpu>

In [434]:
import errno
import time

from paramiko import SSHException
from fabric.network import NetworkError

In [484]:
my_inst.get()
my_inst.status

'ACTIVE'

In [502]:
my_inst.rebuild(nc.images.find(name='test2-CC-Ubuntu16.04-CUDA8'))

<Server: image-builder-gpu>

In [503]:
old_status = my_inst.status
for _ in range(100):
    time.sleep(10)
    my_inst.get()
    if my_inst.status == 'ACTIVE':
        print()
        break
    old_status = my_inst.status
    print('.', end='')

print('{} {} -> {}'.format(datetime.datetime.now().isoformat(), old_status, my_inst.status))

..............................
2017-04-09T00:31:23.763068 REBUILD -> ACTIVE


In [504]:
while True:
    try:
        with fapi.settings(abort_on_prompts=True):
            fapi.run('', quiet=True)
        break
    except OSError as e:
        if e.errno == errno.ENETUNREACH:
            pass
        raise
    except (NetworkError, SSHException):
        pass
    print('.', end='')
    time.sleep(10)

................

In [506]:
fapi.run('touch /home/cc/.cloud-warnings.skip', quiet=True)
fapi.run('nvidia-smi')

[129.114.108.248] run: nvidia-smi
[129.114.108.248] out: /bin/bash: nvidia-smi: command not found
[129.114.108.248] out: 



'/bin/bash: nvidia-smi: command not found'

In [342]:
print(f'glance --os-auth-url {sess.auth.auth_url} \
--os-user-id {sess.get_user_id()} \
--os-username {sess.auth.auth_ref.username} \
--os-auth-token {sess.get_token()} \
--os-project-id {sess.get_project_id()} \
image-list')

glance --os-auth-url https://chi.tacc.chameleoncloud.org:5000/v2.0 --os-user-id 968303d4fb88415e8d8ed3c97f744958 --os-username ntimkovi --os-auth-token 167e8fc82ca742f8b439f4e99c61cff7 --os-project-id 975c0a94b784483a885f4503f70af655 image-list


In [394]:
147456 / 1024 / 16

9.0

In [489]:
image_name = 'test4-CC-Ubuntu16.04-CUDA8'
image = nc.images.find(name=image_name)
with fcm.shell_env(**rc):#, fapi.cd('/home/cc/build'):
    out = fapi.run('glance image-download --file {}.qcow2 {}'.format(image_name, image.id))
#                    '--file /tmp/tmp.kTi8NRm9CS/common/CC-Ubuntu16.04.qcow2')

[129.114.108.248] run: glance image-download --file test4-CC-Ubuntu16.04-CUDA8.qcow2 99ca5101-de22-4fe3-bbf8-a27c85ebc0f3


In [494]:
# fapi.run('mkdir mount')
fapi.sudo(f'guestmount -a /home/cc/{image_name}.qcow2 -m /dev/sdb1 /home/cc/mount')

[129.114.108.248] sudo: guestmount -a /home/cc/test4-CC-Ubuntu16.04-CUDA8.qcow2 -m /dev/sdb1 /home/cc/mount
[129.114.108.248] out: sudo: unable to resolve host image-builder-gpu
[129.114.108.248] out: libguestfs: error: mount_options: mount_options_stub: /dev/sdb1: No such file or directory
[129.114.108.248] out: guestmount: '/dev/sdb1' could not be mounted.
[129.114.108.248] out: guestmount: Did you mean to mount one of these filesystems?
[129.114.108.248] out: guestmount: 	/dev/sda1 (ext4)
[129.114.108.248] out: 



"sudo: unable to resolve host image-builder-gpu\r\nlibguestfs: error: mount_options: mount_options_stub: /dev/sdb1: No such file or directory\r\nguestmount: '/dev/sdb1' could not be mounted.\r\nguestmount: Did you mean to mount one of these filesystems?\r\nguestmount: \t/dev/sda1 (ext4)"

In [492]:
from fabric import api as fapi
from fabric import context_managers as fcm

fapi.env.warn_only = True
fapi.env.use_ssh_config = True
user = fapi.env.user = 'cc'
host = fapi.env.host_string = '129.114.108.248'

image_loc = '/tmp/tmp.HXN40mQ9SY/common/CC-Ubuntu16.04-CUDA8.qcow2'

with fcm.shell_env(**rc):#, fapi.cd('/home/cc/build'):
    out = fapi.run('glance image-create '
                   '--name "test8-CC-Ubuntu16.04-CUDA8" '
                   '--disk-format qcow2 '
                   '--container-format bare '
                   '--file {}'.format(image_loc))
#                    '--file /tmp/tmp.kTi8NRm9CS/common/CC-Ubuntu16.04.qcow2')

[129.114.108.248] run: glance image-create --name "test8-CC-Ubuntu16.04-CUDA8" --disk-format qcow2 --container-format bare --file /tmp/tmp.HXN40mQ9SY/common/CC-Ubuntu16.04-CUDA8.qcow2
[129.114.108.248] out: +------------------+--------------------------------------+
[129.114.108.248] out: | Property         | Value                                |
[129.114.108.248] out: +------------------+--------------------------------------+
[129.114.108.248] out: | checksum         | 073bc8ef06e14183143750ee1dcfa550     |
[129.114.108.248] out: | container_format | bare                                 |
[129.114.108.248] out: | created_at       | 2017-04-09T03:54:35Z                 |
[129.114.108.248] out: | disk_format      | qcow2                                |
[129.114.108.248] out: | id               | d21d59bb-26d9-470f-b1a6-1de2b8789874 |
[129.114.108.248] out: | min_disk         | 0                                    |
[129.114.108.248] out: | min_ram          | 0                        

In [493]:
image_data = {}
for line in out.splitlines():
    parts = [p.strip() for p in line.strip(' |\n\t').split('|')]
    if len(parts) != 2:
        continue
    key, value = parts
    if key == 'Property':
        continue
    image_data[key] = value
    
image_data['id']

'd21d59bb-26d9-470f-b1a6-1de2b8789874'

In [495]:
my_inst.rebuild(image_data['id'])

<Server: image-builder-gpu>

In [496]:
my_inst.get()
my_inst.status

'REBUILD'

In [497]:
my_inst.image

{'id': 'd21d59bb-26d9-470f-b1a6-1de2b8789874',
 'links': [{'href': 'http://chi.tacc.chameleoncloud.org:8774/975c0a94b784483a885f4503f70af655/images/d21d59bb-26d9-470f-b1a6-1de2b8789874',
   'rel': 'bookmark'}]}

In [498]:
nc.images.find(id=my_inst.image['id'])

<Image: test8-CC-Ubuntu16.04-CUDA8>

In [452]:
my_inst.reboot()

()

In [479]:
fapi.run('lspci | grep -i nvid')

[129.114.108.248] run: lspci | grep -i nvid
[129.114.108.248] out: 03:00.0 3D controller: NVIDIA Corporation Device 15f8 (rev a1)
[129.114.108.248] out: 82:00.0 3D controller: NVIDIA Corporation Device 15f8 (rev a1)
[129.114.108.248] out: 



'03:00.0 3D controller: NVIDIA Corporation Device 15f8 (rev a1)\r\n82:00.0 3D controller: NVIDIA Corporation Device 15f8 (rev a1)'